In [1]:
from collections import defaultdict

In [2]:
import pandas as pd
from flask import Flask
from flask import request
from flask import render_template

app = Flask(__name__, template_folder = 'templates')

data = pd.read_csv('/Users/shubhamp/Sjsu/CMPE 255/Project/fifa19_dataset/fifa19_train.csv')

df2=data.groupby('player_name', group_keys=False, as_index=False).apply(lambda x: x.loc[x.overall.idxmax()])

my_df=df2[['quality','position','intl_rep','pace', 'pace_acceleration', 'pace_sprint_speed',
       'dribbling', 'drib_agility', 'drib_balance', 'drib_reactions',
       'drib_ball_control', 'drib_dribbling', 'drib_composure', 'shooting',
       'shoot_positioning', 'shoot_finishing', 'shoot_shot_power',
       'shoot_long_shots', 'shoot_volleys', 'shoot_penalties', 'passing',
       'pass_vision', 'pass_crossing', 'pass_free_kick', 'pass_short',
       'pass_long', 'pass_curve', 'defending', 'def_interceptions',
       'def_heading', 'def_marking', 'def_stand_tackle', 'def_slid_tackle',
       'physicality', 'phys_jumping', 'phys_stamina', 'phys_strength',
       'phys_aggression', 'gk_diving', 'gk_reflexes', 'gk_handling',
       'gk_speed', 'gk_kicking', 'gk_positoning', 'cb', 'rb', 'lb', 'rwb',
       'lwb', 'cdm', 'cm', 'rm', 'lm', 'cam', 'cf', 'rf', 'lf', 'rw', 'lw',
       'st','pref_foot', 'att_workrate',
       'def_workrate', 'weak_foot', 'skill_moves']]
    

my_df = my_df.fillna(0)
my_df['quality']=pd.factorize(my_df['quality'])[0]
my_df['position']=pd.factorize(my_df['position'])[0]
my_df['pref_foot']=pd.factorize(my_df['pref_foot'])[0]
my_df['def_workrate']=pd.factorize(my_df['def_workrate'])[0]
my_df['att_workrate']=pd.factorize(my_df['att_workrate'])[0]
my_df = my_df.astype(int)

from sklearn.preprocessing import StandardScaler
scaled = StandardScaler()
X = scaled.fit_transform(my_df)

from sklearn.neighbors import NearestNeighbors
recommendations = NearestNeighbors(n_neighbors=6,algorithm='ball_tree')
recommendations.fit(X)

player_index = recommendations.kneighbors(X)[1]

/anaconda3/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4315: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  stacked_values = np.vstack(map(np.asarray, values))


In [3]:
my_df.columns.shape

(65,)

In [4]:
import pickle
filename = 'player_index'
outfile = open(filename,'wb')
pickle.dump( player_index,outfile)
outfile.close()

In [5]:
import pandas as pd
from flask import Flask
from flask import request
from flask import render_template


#### Start overall score code##

In [6]:
###ankit code###
import numpy as np
import itertools

from sklearn.preprocessing import LabelBinarizer,LabelEncoder
from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras import utils

from sklearn.preprocessing import add_dummy_feature
from keras.losses import mean_squared_error,mean_absolute_error
from keras.optimizers import RMSprop
from keras.wrappers.scikit_learn import KerasRegressor
import tensorflow as tf
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

df=pd.read_csv("fifa19_train.csv")
my_df=df[['quality','age','height', 'weight','position','intl_rep','pace', 'pace_acceleration', 'pace_sprint_speed',
       'dribbling', 'drib_agility', 'drib_balance', 'drib_reactions',
       'drib_ball_control', 'drib_dribbling', 'drib_composure', 'shooting',
       'shoot_positioning', 'shoot_finishing', 'shoot_shot_power',
       'shoot_long_shots', 'shoot_volleys', 'shoot_penalties', 'passing',
       'pass_vision', 'pass_crossing', 'pass_free_kick', 'pass_short',
       'pass_long', 'pass_curve', 'defending', 'def_interceptions',
       'def_heading', 'def_marking', 'def_stand_tackle', 'def_slid_tackle',
       'physicality', 'phys_jumping', 'phys_stamina', 'phys_strength',
       'phys_aggression', 'gk_diving', 'gk_reflexes', 'gk_handling',
       'gk_speed', 'gk_kicking', 'gk_positoning','pref_foot', 'att_workrate',
       'def_workrate', 'weak_foot', 'skill_moves','cb', 'rb', 'lb', 'rwb',
       'lwb', 'cdm', 'cm', 'rm', 'lm', 'cam', 'cf', 'rf', 'lf', 'rw', 'lw',
       'st']]
my_df=my_df.fillna(0)
my_df['quality'],uni_qual=pd.factorize(my_df['quality'])
my_df['position'],uni_pos=pd.factorize(my_df['position'])
my_df['pref_foot'],uni_foot=pd.factorize(my_df['pref_foot'])
my_df['def_workrate'],uni_def=pd.factorize(my_df['def_workrate'])
my_df['att_workrate'],uni_att=pd.factorize(my_df['att_workrate'])
my_df=my_df.astype(int)
y=df['overall']
X_train, X_test, y_train, y_test = train_test_split(my_df, y)
# define base model
a,b = X_train.shape
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(b, input_dim=b, kernel_initializer='normal', activation='relu'))
	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model


estimator = KerasRegressor(build_fn=baseline_model, epochs=200, batch_size=100, verbose=1)
results=estimator.fit(X_train,y_train)

y_pred = estimator.predict(X_test)


###end keras

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Epoch 1/200
12710/12710 [==============================] - 0s 24us/step - loss: 171.2695
Epoch 2/200
12710/12710 [==============================] - 0s 11us/step - loss: 12.1794
Epoch 3/200
12710/12710 [==============================] - 0s 12us/step - loss: 8.8255
Epoch 4/200
12710/12710 [==============================] - 0s 10us/step - loss: 5.9360
Epoch 5/200
12710/12710 [==============================] - 0s 10us/step - loss: 4.1980
Epoch 6/200
12710/12710 [==============================] - 0s 10us/step - loss: 3.4151
Epoch 7/200
12710/12710 [==============================] - 0s 10us/step - loss: 3.0052
Epoch 8/200
12710/12710 [==============================] - 0s 10us/step - loss: 2.7209
Epoch 9/200
12710/12710 [==============================] - 0s 10us/step - loss: 2.4763
Epoch 10/200
12710/12710 [==============================] - 0s 9us/step - loss: 2.3396
Epoch 1

12710/12710 [==============================] - 0s 9us/step - loss: 0.9196
Epoch 89/200
12710/12710 [==============================] - 0s 10us/step - loss: 0.8254
Epoch 90/200
12710/12710 [==============================] - 0s 11us/step - loss: 0.8390
Epoch 91/200
12710/12710 [==============================] - 0s 9us/step - loss: 0.8757
Epoch 92/200
12710/12710 [==============================] - 0s 9us/step - loss: 0.9610
Epoch 93/200
12710/12710 [==============================] - 0s 9us/step - loss: 0.8822
Epoch 94/200
12710/12710 [==============================] - 0s 9us/step - loss: 0.8710
Epoch 95/200
12710/12710 [==============================] - 0s 9us/step - loss: 0.8782
Epoch 96/200
12710/12710 [==============================] - 0s 9us/step - loss: 0.9000
Epoch 97/200
12710/12710 [==============================] - 0s 9us/step - loss: 0.9118
Epoch 98/200
12710/12710 [==============================] - 0s 9us/step - loss: 1.0249
Epoch 99/200
12710/12710 [============================

12710/12710 [==============================] - 0s 9us/step - loss: 0.7977
Epoch 181/200
12710/12710 [==============================] - 0s 9us/step - loss: 0.8195
Epoch 182/200
12710/12710 [==============================] - 0s 9us/step - loss: 0.8239
Epoch 183/200
12710/12710 [==============================] - 0s 11us/step - loss: 0.8793
Epoch 184/200
12710/12710 [==============================] - 0s 14us/step - loss: 0.8182
Epoch 185/200
12710/12710 [==============================] - 0s 17us/step - loss: 0.8197
Epoch 186/200
12710/12710 [==============================] - 0s 13us/step - loss: 0.8793
Epoch 187/200
12710/12710 [==============================] - 0s 14us/step - loss: 0.8834
Epoch 188/200
12710/12710 [==============================] - 0s 13us/step - loss: 0.8125
Epoch 189/200
12710/12710 [==============================] - 0s 12us/step - loss: 0.8021
Epoch 190/200
12710/12710 [==============================] - 0s 13us/step - loss: 0.7712
Epoch 191/200
12710/12710 [===========

In [7]:
import pickle
filename = 'overall_model'
outfile = open(filename,'wb')
pickle.dump(estimator,outfile)
outfile.close()

In [8]:
# testdata = pd.read_csv("fifa19_test.csv")
# my_dftest=testdata[['quality','age','height', 'weight','position','intl_rep','pace', 'pace_acceleration', 'pace_sprint_speed',
#        'dribbling', 'drib_agility', 'drib_balance', 'drib_reactions',
#        'drib_ball_control', 'drib_dribbling', 'drib_composure', 'shooting',
#        'shoot_positioning', 'shoot_finishing', 'shoot_shot_power',
#        'shoot_long_shots', 'shoot_volleys', 'shoot_penalties', 'passing',
#        'pass_vision', 'pass_crossing', 'pass_free_kick', 'pass_short',
#        'pass_long', 'pass_curve', 'defending', 'def_interceptions',
#        'def_heading', 'def_marking', 'def_stand_tackle', 'def_slid_tackle',
#        'physicality', 'phys_jumping', 'phys_stamina', 'phys_strength',
#        'phys_aggression', 'gk_diving', 'gk_reflexes', 'gk_handling',
#        'gk_speed', 'gk_kicking', 'gk_positoning','pref_foot', 'att_workrate',
#        'def_workrate', 'weak_foot', 'skill_moves','cb', 'rb', 'lb', 'rwb',
#        'lwb', 'cdm', 'cm', 'rm', 'lm', 'cam', 'cf', 'rf', 'lf', 'rw', 'lw',
#        'st']]
# my_dftest=my_dftest.fillna(0)
# #my_dftest['quality']=pd.factorize(my_dftest['quality'])[0]
# #my_dftest['position']=pd.factorize(my_dftest['position'])[0]
# #my_dftest['pref_foot']=pd.factorize(my_dftest['pref_foot'])[0]
# #my_dftest['def_workrate']=pd.factorize(my_dftest['def_workrate'])[0]
# #my_dftest['att_workrate']=pd.factorize(my_dftest['att_workrate'])[0]
# #my_dftest = my_dftest.astype(int)
# ytest=testdata['overall']

In [9]:
# list(uniques).index('a')

In [10]:
# def factorize_column(dftest, column, uniques):
#     dftest[column] = list(uniques).index(dftest[column])
#     return dftest

# my_dftest = my_dftest.apply(factorize_column, axis=1, args =('quality', uni_qual))
# my_dftest = my_dftest.apply(factorize_column, axis=1, args =('position', uni_pos))
# my_dftest = my_dftest.apply(factorize_column, axis=1, args = ('pref_foot', uni_foot))
# my_dftest = my_dftest.apply(factorize_column, axis=1, args = ('def_workrate', uni_def))
# my_dftest = my_dftest.apply(factorize_column, axis=1, args = ('att_workrate', uni_att))

In [11]:
# y_test_predictions = estimator.predict(my_dftest)

In [12]:
# from sklearn.metrics import r2_score, mean_squared_error
# def evaluate(model,y_predict, y_test):
#     print(model)
#     r2=r2_score(y_test,y_predict)
#     rms= mean_squared_error(y_test,y_predict)
#     print("R2 score:",r2)
#     print("RMSE score:",rms)
#     return r2,rms

In [13]:
# evaluate("keras",y_test_predictions,ytest)

#### end overall code##

In [14]:
df2 = df2.fillna(0);

In [15]:
test = df2.iloc[1,:].to_dict()

In [16]:
df=pd.read_csv("fifa19_train.csv")
temp=df[['quality','age','height', 'weight','position','intl_rep','pace', 'pace_acceleration', 'pace_sprint_speed',
       'dribbling', 'drib_agility', 'drib_balance', 'drib_reactions',
       'drib_ball_control', 'drib_dribbling', 'drib_composure', 'shooting',
       'shoot_positioning', 'shoot_finishing', 'shoot_shot_power',
       'shoot_long_shots', 'shoot_volleys', 'shoot_penalties', 'passing',
       'pass_vision', 'pass_crossing', 'pass_free_kick', 'pass_short',
       'pass_long', 'pass_curve', 'defending', 'def_interceptions',
       'def_heading', 'def_marking', 'def_stand_tackle', 'def_slid_tackle',
       'physicality', 'phys_jumping', 'phys_stamina', 'phys_strength',
       'phys_aggression', 'gk_diving', 'gk_reflexes', 'gk_handling',
       'gk_speed', 'gk_kicking', 'gk_positoning','pref_foot', 'att_workrate',
       'def_workrate', 'weak_foot', 'skill_moves','cb', 'rb', 'lb', 'rwb',
       'lwb', 'cdm', 'cm', 'rm', 'lm', 'cam', 'cf', 'rf', 'lf', 'rw', 'lw',
       'st']]

In [17]:
# columnList = defaultdict(list)
# for col in temp.columns:
#     columnList[col]+=[temp[col][0]]
# columnList
# predict_for=pd.DataFrame(columnList)
# predict_for = predict_for.fillna(0)
# predict_for['quality']=list(uni_qual).index(predict_for['quality'][0])
# predict_for['position']=list(uni_pos).index(predict_for['position'][0])
# predict_for['pref_foot']=list(uni_foot).index(predict_for['pref_foot'][0])
# predict_for['def_workrate']=list(uni_def).index(predict_for['def_workrate'][0])
# predict_for['att_workrate']=list(uni_att).index(predict_for['att_workrate'][0])
# #predict_for.fillna(0)

# predict_for#=predict_for.astype(int)

In [18]:
# list(predict_for.values)

In [19]:
#predict_for['quality'][0]

In [20]:
#predict_for

In [21]:
#list(uni_qual).index(predict_for['quality'][0])

In [22]:
# if request.form.get('player2')==None:
#list(uni_qual)

In [23]:
my_df.columns.shape

(68,)

In [24]:
df2.columns.shape

(96,)

In [25]:
df2.head(1)

,Unnamed: 0,player_id,player_name,player_extended_name,quality,revision,origin,overall,club,league,...,ps4_max,ps4_prp,xbox_last,xbox_min,xbox_max,xbox_prp,pc_last,pc_min,pc_max,pc_prp
0,12594,13052,Aabid,Rayane Aabid,Bronze,Normal,0,62,AS Béziers,Domino’s Ligue 2,...,10000,0,750,150,10000,6,200,150,10000,0


In [ ]:
@app.route('/', methods=['GET', 'POST'])
# @app.route('/index', methods=['GET', 'POST'])
def index():
    if request.method == 'GET':
        return render_template('index.html')
    else:
        if request.form.get('Player Name')==None:
            if request.form.get('player2')!=None:
                text1 = request.form['player2']
                def get_index(x):
                    return df2[df2['player_name']==x].index.tolist()[0]
                index=get_index(text1)
                my_dict=df2.iloc[index,:].to_dict()
                return render_template('index.html',posts6=my_dict["position"],posts7=my_dict["intl_rep"],posts8=my_dict["quality"],posts9=my_dict["pace"],posts10=my_dict["pace_acceleration"],posts11=my_dict["pace_sprint_speed"],posts12=my_dict["dribbling"],posts13=my_dict["drib_agility"],posts14=my_dict["drib_balance"],posts15=my_dict["drib_reactions"],posts16=my_dict["drib_ball_control"],posts17=my_dict["drib_dribbling"],posts18=my_dict["drib_composure"],posts19=my_dict["shooting"],posts20=my_dict["shoot_positioning"],posts21=my_dict["shoot_finishing"],posts22=my_dict["shoot_shot_power"],posts23=my_dict["shoot_long_shots"],posts24=my_dict["shoot_volleys"],posts25=my_dict["shoot_penalties"],posts26=my_dict["passing"],posts27=my_dict["pass_vision"],posts28=my_dict["pass_crossing"],posts29=my_dict["pass_free_kick"],posts30=my_dict["pass_short"],posts31=my_dict["pass_long"],posts32=my_dict["pass_curve"],posts33=my_dict["defending"],posts34=my_dict["def_interceptions"],posts35=my_dict["def_heading"],posts36=my_dict["def_marking"],posts37=my_dict["def_stand_tackle"],posts38=my_dict["def_slid_tackle"],posts39=my_dict["physicality"],posts40=my_dict["phys_jumping"],posts41=my_dict["phys_stamina"],posts42=my_dict["phys_strength"],posts43=my_dict["phys_aggression"],posts44=my_dict["gk_diving"],posts45=my_dict["gk_reflexes"],posts46=my_dict["gk_handling"],posts47=my_dict["gk_speed"],posts48=my_dict["gk_kicking"],posts49=my_dict["gk_positoning"],posts50=my_dict["pref_foot"],posts51=my_dict["att_workrate"],posts52=my_dict["def_workrate"],posts53=my_dict["weak_foot"],posts54=my_dict["skill_moves"],posts55=my_dict["cb"],posts56=my_dict["rb"],posts57=my_dict["lb"],posts58=my_dict["rwb"],posts59=my_dict["lwb"],posts60=my_dict["cdm"],posts61=my_dict["cm"],posts62=my_dict["rm"],posts63=my_dict["lm"],posts64=my_dict["cam"],posts65=my_dict["cf"],posts66=my_dict["rf"],posts67=my_dict["lf"],posts68=my_dict["rw"],posts69=my_dict["lw"],posts70=my_dict["st"],posts71=my_dict["age"],posts72=my_dict["height"],posts73=my_dict["weight"])
            else:
                #position = request.form.get('position')
                
                #data = request.form.to_dict()
                filename = 'overall_model'
                infile = open(filename,'rb')
                model = pickle.load(infile)
                infile.close()
                
                columnList = defaultdict(list)
                for col in my_df.columns:
                    columnList[col]+=[request.form[col]]
                
                predict_for=pd.DataFrame(columnList)
                print(predict_for.head())
                #print(predict_for.head(1))
                #testt = predict_for
               
                predict_for['quality']=list(uni_qual).index(predict_for['quality'][0])
                predict_for['position']=list(uni_pos).index(predict_for['position'][0])
                predict_for['pref_foot']=list(uni_foot).index(predict_for['pref_foot'][0])
                predict_for['def_workrate']=list(uni_def).index(predict_for['def_workrate'][0])
                predict_for['att_workrate']=list(uni_att).index(predict_for['att_workrate'][0])
                predict_for = predict_for.fillna(0)
                predict_for=predict_for.astype(float)
                #x= scaled.transform(predict_for)

                foverall=model.predict(predict_for)
                foverall=round(float(foverall),2)
                
                return render_template('index.html',score=foverall)

        else:
            text1 = request.form['Player Name']
            filename = 'player_index'
            infile = open(filename,'rb')
            new_dict = pickle.load(infile)
            infile.close()

            def get_index(x):
                search = df2[df2['player_name']==x]
                if search.size==0:
                    return -1
                return search.index.tolist()[0]

            def recommend_me(player):
                index =  get_index(player)
                if index==-1:
                    return ["Player not available"]
                recomm=[]
                for i in new_dict[index][1:]:
                    recomm.append(df2.iloc[i]['player_name'])
                return recomm
            rec=recommend_me(text1)

            #print(rec[2])
            #if len(rec) == 0:
                #return render_template('index.html', posts1="No such player exists",posts2="",posts3="",posts4="",posts5="")
            if len(rec)==1:
                return render_template('index.html', posts1=rec[0],posts2="",posts3="",posts4="",posts5="")
            else:
                #return render_template('index.html',rec)
                return render_template('index.html',posts1=rec[0],posts2=rec[1],posts3=rec[2],posts4=rec[3],posts5=rec[4])
                #return render_template('index.html', posts1=rec[0],posts2=rec[1],posts3=rec[2],posts4=rec[3],posts5=rec[4])

# @app.route('/', methods=['GET', 'POST'])
# def overall():
#     return render_template('index.html')

if __name__ == "__main__":
    app.run()

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/May/2019 16:44:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2019 16:44:59] "GET /js/jquery.min.js HTTP/1.1" 404 -
127.0.0.1 - - [09/May/2019 16:44:59] "GET /js/jquery.easing.min.js HTTP/1.1" 404 -
127.0.0.1 - - [09/May/2019 16:44:59] "GET /js/bootstrap.min.js HTTP/1.1" 404 -
127.0.0.1 - - [09/May/2019 16:44:59] "GET /js/wow.js HTTP/1.1" 404 -
127.0.0.1 - - [09/May/2019 16:44:59] "GET /js/custom.js HTTP/1.1" 404 -
127.0.0.1 - - [09/May/2019 16:44:59] "GET /contactform/contactform.js HTTP/1.1" 404 -
127.0.0.1 - - [09/May/2019 16:44:59] "GET /js/jquery.min.js HTTP/1.1" 404 -
127.0.0.1 - - [09/May/2019 16:44:59] "GET /js/jquery.easing.min.js HTTP/1.1" 404 -
127.0.0.1 - - [09/May/2019 16:44:59] "GET /js/bootstrap.min.js HTTP/1.1" 404 -
127.0.0.1 - - [09/May/2019 16:44:59] "GET /js/wow.js HTTP/1.1" 404 -
127.0.0.1 - - [09/May/2019 16:44:59] "GET /js/custom.js HTTP/1.1" 404 -
127.0.0.1 - - [09/May/2019 16:44:59]

In [ ]:
a = {"name":["1.1"],"id":["2"]}

In [ ]:
pd.DataFrame(list(a.items()))

In [ ]:
X=list(a.items())

In [ ]:
X

In [ ]:
x = pd.DataFrame.from_dict(a, orient='columns')
x["name"][0]

In [ ]:
x = x.astype(float)
x["name"][0]